In [135]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
import numpy as np
from scipy.signal import butter, filtfilt
from scipy.signal import stft

import librosa
import cv2
import pywt
import matplotlib.pyplot as plt


from tqdm import tqdm
import jax.numpy as jnp
import matplotlib.pyplot as plt
from superlets import wavelet_transform, adaptive_superlet_transform

In [136]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt, gc

train = pd.read_csv('/Users/anishpawar/MTech/Sem-2/IVP/Project_PreProc_Final_Report/IVP-EEG-Classification/hms-harmful-brain-activity-classification/train.csv')
print('Train shape', train.shape )
display( train.head(10) )

Train shape (106800, 15)


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0
5,1628180742,5,26.0,353733,5,26.0,2413091605,42516,Seizure,3,0,0,0,0,0
6,1628180742,6,30.0,353733,6,30.0,364593930,42516,Seizure,3,0,0,0,0,0
7,1628180742,7,36.0,353733,7,36.0,3811483573,42516,Seizure,3,0,0,0,0,0
8,1628180742,8,40.0,353733,8,40.0,3388718494,42516,Seizure,3,0,0,0,0,0
9,2277392603,0,0.0,924234,0,0.0,1978807404,30539,GPD,0,0,5,0,1,5


In [137]:
NAMES = ['LL','LP','RP','RR']

FEATS = [['Fp1','F7','T3','T5','O1'],
         ['Fp1','F3','C3','P3','O1'],
         ['Fp2','F8','T4','T6','O2'],
         ['Fp2','F4','C4','P4','O2']]

directory_path = 'EEG_Spectrograms/'
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

In [138]:
import pywt
print("The wavelet functions we can use:")
print(pywt.wavelist())

USE_WAVELET = "None"

The wavelet functions we can use:
['bior1.1', 'bior1.3', 'bior1.5', 'bior2.2', 'bior2.4', 'bior2.6', 'bior2.8', 'bior3.1', 'bior3.3', 'bior3.5', 'bior3.7', 'bior3.9', 'bior4.4', 'bior5.5', 'bior6.8', 'cgau1', 'cgau2', 'cgau3', 'cgau4', 'cgau5', 'cgau6', 'cgau7', 'cgau8', 'cmor', 'coif1', 'coif2', 'coif3', 'coif4', 'coif5', 'coif6', 'coif7', 'coif8', 'coif9', 'coif10', 'coif11', 'coif12', 'coif13', 'coif14', 'coif15', 'coif16', 'coif17', 'db1', 'db2', 'db3', 'db4', 'db5', 'db6', 'db7', 'db8', 'db9', 'db10', 'db11', 'db12', 'db13', 'db14', 'db15', 'db16', 'db17', 'db18', 'db19', 'db20', 'db21', 'db22', 'db23', 'db24', 'db25', 'db26', 'db27', 'db28', 'db29', 'db30', 'db31', 'db32', 'db33', 'db34', 'db35', 'db36', 'db37', 'db38', 'dmey', 'fbsp', 'gaus1', 'gaus2', 'gaus3', 'gaus4', 'gaus5', 'gaus6', 'gaus7', 'gaus8', 'haar', 'mexh', 'morl', 'rbio1.1', 'rbio1.3', 'rbio1.5', 'rbio2.2', 'rbio2.4', 'rbio2.6', 'rbio2.8', 'rbio3.1', 'rbio3.3', 'rbio3.5', 'rbio3.7', 'rbio3.9', 'rbio4.4', 'rbio5.5'

In [139]:
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def denoise(x, wavelet='haar', level=1):    
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * maddest(coeff[-level])

    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

    ret=pywt.waverec(coeff, wavelet, mode='per')
    
    return ret

In [140]:
root = '/Users/anishpawar/MTech/Sem-2/IVP/Project_PreProc_Final_Report/IVP-EEG-Classification/hms-harmful-brain-activity-classification'

metadata = pd.read_csv(f'{root}/train.csv')
metadata

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,351917269,6,12.0,2147388374,6,12.0,4195677307,10351,LRDA,0,0,0,3,0,0
106796,351917269,7,14.0,2147388374,7,14.0,290896675,10351,LRDA,0,0,0,3,0,0
106797,351917269,8,16.0,2147388374,8,16.0,461435451,10351,LRDA,0,0,0,3,0,0
106798,351917269,9,18.0,2147388374,9,18.0,3786213131,10351,LRDA,0,0,0,3,0,0


In [141]:
def getWindowsfromEEG(eeg_id):
    # print(eeg_id)
    # eeg_id = 1628180742
    # eeg_df = metadata[metadata["eeg_id"] == eeg_id].reset_index(drop=True)
    eeg = pd.read_parquet(f'{root}/train_eegs/{eeg_id}.parquet')

    fs = 200
    eeg["time"] = eeg.index / fs
    eeg.set_index("time", inplace=True)
    eeg.index

    subSet = metadata[metadata['eeg_id']==eeg_id]
    # print(subSet)
    offSets = [subSet['eeg_sub_id'].to_numpy(),subSet['eeg_label_offset_seconds'].to_numpy()]
    offSets = np.array(offSets)
    


    toRet = []
    for i in offSets[1]:
        toRet.append(eeg.loc[i+20:i+30])
    # toRet = np.array(toRet)
    return offSets[0],toRet,eeg


In [142]:
from scipy.signal import butter, filtfilt

def apply_lpf(data, sampling_rate=200, cutoff_freq=10, order=5):

    nyquist_freq = 0.5 * sampling_rate
    normal_cutoff = cutoff_freq / nyquist_freq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = filtfilt(b, a, data)

    return filtered_data

In [143]:
# import librosa

# def spectrogram_from_eeg(eeg_id, display=False):
    

#     offSets,sliced_eegs,OG_eeg = getWindowsfromEEG(eeg_id)

#     img = np.zeros((128,128,4),dtype='float32')
    
#     if display: plt.figure(figsize=(10,7))

#     retSpecs = []
#     for idx,eeg in enumerate(sliced_eegs):
#         # print(eeg.shape)
#         signals = []
#         for k in range(4):
#             COLS = FEATS[k]
#             # print(COLS)
#             for kk in range(4):
            
#                 # COMPUTE PAIR DIFFERENCES
#                 x = eeg[COLS[kk]] - eeg[COLS[kk+1]]

#                 # FILL NANS
#                 m = np.nanmean(x)
#                 if np.isnan(x).mean()<1: x = np.nan_to_num(x,nan=m)
#                 else: x[:] = 0

#                 # DENOISE
#                 if USE_WAVELET=='LPF10':
#                     # print(USE_WAVELET)
#                     x = apply_lpf(x,cutoff_freq=10)
#                 elif USE_WAVELET=='LPF20':
#                     # print(USE_WAVELET)
#                     x = apply_lpf(x,cutoff_freq=20)
#                 elif USE_WAVELET=='None':
#                     # print(USE_WAVELET)
#                     x = x
#                 else:
#                     x = denoise(x, wavelet=USE_WAVELET)
                
#                 signals.append(x)

#                 # RAW SPECTROGRAM
#                 mel_spec = librosa.feature.melspectrogram(y=x, sr=200, hop_length=len(x)//128, 
#                     n_fft=1024, n_mels=128, fmin=0, fmax=20, win_length=128)

#                 # LOG TRANSFORM
#                 width = (mel_spec.shape[1]//32)*32
#                 mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).astype(np.float32)[:,:width]

#                 # STANDARDIZE TO -1 TO 1
#                 mel_spec_db = (mel_spec_db+40)/40 
#                 img[:,:,k] += mel_spec_db
                    
#             # AVERAGE THE 4 MONTAGE DIFFERENCES
#             img[:,:,k] /= 4.0
#         np.save(f'Processed_Specs/{eeg_id}_spec-{idx}-{USE_WAVELET}.npy',img)
#         # print(idx)
#         # retSpecs.append(img)
#     # print(retSpecs)
#     # retSpecs = np.array(retSpecs)
#     # return retSpecs


In [144]:
import cv2

In [145]:
import librosa

def spectrogram_from_eeg(eeg_id, display=False,specMethod="MEL",USE_WAVELET="None"):
    

    offSets,sliced_eegs,OG_eeg = getWindowsfromEEG(eeg_id)

    img = np.zeros((128,128,4),dtype='float32')
    
    if display: plt.figure(figsize=(10,7))

    retSpecs = []
    for idx,eeg in enumerate(sliced_eegs):
        # print(eeg.shape)
        signals = []
        for k in range(4):
            COLS = FEATS[k]
            # print(COLS)
            for kk in range(4):
            
                # COMPUTE PAIR DIFFERENCES
                x = eeg[COLS[kk]] - eeg[COLS[kk+1]]

                # FILL NANS
                m = np.nanmean(x)
                if np.isnan(x).mean()<1: x = np.nan_to_num(x,nan=m)
                else: x[:] = 0

                # DENOISE
                if USE_WAVELET=='LPF10':
                    # print(USE_WAVELET)
                    x = apply_lpf(x,cutoff_freq=10)
                elif USE_WAVELET=='LPF20':
                    # print(USE_WAVELET)
                    x = apply_lpf(x,cutoff_freq=20)
                elif USE_WAVELET=='None':
                    # print(USE_WAVELET)
                    x = x
                else:
                    x = denoise(x, wavelet=USE_WAVELET)
                
                signals.append(x)

                # RAW SPECTROGRAM
                if specMethod == 'MEL':
                    mel_spec = librosa.feature.melspectrogram(y=x, sr=200, hop_length=len(x)//128, 
                        n_fft=1024, n_mels=128, fmin=0, fmax=20, win_length=128)

                    # LOG TRANSFORM
                    width = (mel_spec.shape[1]//32)*32
                    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).astype(np.float32)[:,:width]

                    # STANDARDIZE TO -1 TO 1
                    mel_spec_db = (mel_spec_db+40)/40 
                    img[:,:,k] += mel_spec_db

                if specMethod == 'SL':
                    # print('OK')
                    
                    freqs = jnp.linspace(1, 25, 25)

                    scalogram = adaptive_superlet_transform(x, freqs, sampling_freq=200, 
                                            base_cycle=5, min_order=5, 
                                            max_order=30, mode="add")
                    
                    image = np.abs(scalogram)**2
                    image_normalized = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
                    image_to_show = np.uint8(image_normalized)

                    
                    # image_colored = cv2.applyColorMap(image_to_show, cv2.COLORMAP_JET)

                    # window_name = f"Base cycles: {base_cycle}, Orders: {min_order}-{max_order}"
                    # cv2.imwrite(f'superlet-{i}.png',image_to_show)
                    img[:,:,k] += cv2.resize(image_to_show, (128, 128))

                if specMethod == 'CWT':

                    def spectrogram_cwt(signal, wavelet='morl', scales=np.arange(1, 128)):
                        coeffs, freqs = pywt.cwt(signal, scales, wavelet)
                        power = (np.abs(coeffs)) ** 2
                        return power, freqs


                    fs = 200
                    t = np.arange(0, 10, 1/fs)
                    

                    scales = np.arange(1, 32)


                    power, freqs = spectrogram_cwt(x, scales=scales)


                    power_normalized = ((power - power.min()) / (power.max() - power.min()) * 255).astype(np.uint8)



                    spectrogram_image_resized = cv2.resize(power_normalized, (128, 128))

                    img[:,:,k] += spectrogram_image_resized
                    
            # AVERAGE THE 4 MONTAGE DIFFERENCES
            img[:,:,k] /= 4.0
        if idx==0:
            np.save(f'PS2/{eeg_id}_spec-{idx}-{specMethod}-{USE_WAVELET}.npy',img)
        # print(idx)
        # retSpecs.append(img)
    # print(retSpecs)
    # retSpecs = np.array(retSpecs)
    # return retSpecs


In [151]:
!rm -r PS2

In [152]:
!mkdir PS2

In [153]:
waveletMethods = ['None','LPF10','LPF20','haar','db8']
specMethods = ['MEL',"SL","CWT"]

In [154]:
eeg_id = 1628180742

for wavelet in waveletMethods:
    imgs = spectrogram_from_eeg(eeg_id,specMethod="MEL",USE_WAVELET=wavelet)

In [94]:
def plotSpecs(spec):
    Labels  = ['LL','LP','RP','RR']
    fig = plt.figure(figsize=(10, 10)) 
    for i in range(1,5):

    
        fig.add_subplot(2,2,i )
        plt.title(f'{Labels[i-1]}', fontsize=12)
        plt.imshow(spec[:,:,i-1])
        plt.axis('off')

In [95]:
plotSpecs(imgs[0])

TypeError: 'NoneType' object is not subscriptable

In [ ]:
from tqdm import tqdm

In [ ]:
%%time
PATH = 'dataset/train_eegs/'
DISPLAY = 4
EEG_IDS = [1628180742]
# train.eeg_id.unique()
all_eegs = {}

for eeg_id in tqdm(EEG_IDS):
        # if (i%100==0)&(i!=0): print(i,', ',end='')
        spectrogram_from_eeg(eeg_id)

100%|██████████| 1/1 [00:01<00:00,  1.19s/it]

CPU times: user 7.05 s, sys: 390 ms, total: 7.44 s
Wall time: 1.2 s
